In [181]:
import hashlib
import random
import json
import logging





# Sha256 hash function

(Bitcoin uses 2 round of sha256)

In [2]:
def sha256(message):
    return hashlib.sha256(message.encode('ascii')).hexdigest()

# Mining block hashes with respect to the difficulty

difficulty = number of leading 0's, simpler than bitcoin's, cant scale linearly <br>
nonce = the parameter to change the hash of the block,can be any arbitrary 32bit number

In [171]:
def mine(message, difficulty=1):
    
    nonce = 0
    prefix = '0' * difficulty
    while True:
        digest = sha256(message + str(nonce))
        if digest.startswith(prefix):
            return digest
        nonce = random.randint(0, 2147483647)

# Wallet
public key = address to receive transactions <br>
private key = key to sign transactions <br>
Bitcoin uses Eliptic Curve Manipulation, im using RSA instead also bitcoin wallets are more sophisticated to ensure better privacy and security.

In [275]:
import binascii
import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5


class Wallet(object):
    
    utxo = []
    
    def __init__(self):
        random_gen = Crypto.Random.new().read
        self._private_key = RSA.generate(1024, random_gen)
        self._public_key = self._private_key.publickey()
        self._signer = PKCS1_v1_5.new(self._private_key)
        #self.address = binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')
        
    @property
    def address(self):
        return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')
    
    def sign(self, message):

        h = SHA.new(message.encode('utf8'))
        return binascii.hexlify(self._signer.sign(h)).decode('ascii')
    
    

    
    def compute_balance(self, transactions):
        
        balance = 0
        
        #address = binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')
        
        for tx in transactions:

            for txin in tx.inputs:
                if txin.parent_output.address == self.address:
                    balance -= txin.parent_output.amount

            for txout in tx.outputs:
                if txout.address == self.address:
                    balance += txout.amount
                    
        return balance
        

In [276]:
def verify_signature(wallet_address, message, signature):

    pubkey = RSA.importKey(binascii.unhexlify(wallet_address))
    verifier = PKCS1_v1_5.new(pubkey)
    h = SHA.new(message.encode('utf8'))
    return verifier.verify(h, binascii.unhexlify(signature))


In [277]:
class TransactionInput(object):
    
    def __init__(self, tx, txout_index):
        self.prev_tx = tx
        self.prev_txout_index = txout_index
        #Txin-script length	
        #Txin-script / scriptSig	
        #sequence_no
        
    def to_dict(self):
        d = {
            'transaction': self.prev_tx.tx_hash(),
            'output_index': self.prev_txout_index
        }
        return d
    
    @property
    def parent_output(self):
        return self.prev_tx.outputs[self.prev_txout_index]
    
    

In [278]:
class TransactionOutput(object):

    def __init__(self, address, amount):
        self.address = address
        self.amount = amount
        
    def to_dict(self):
        d = {
            'address': self.address,
            'amount': self.amount
        }
        return d

In [279]:
class Transaction(object):
    
    def __init__(self, sender, inputs, outputs):
        
        self.in_count = len(inputs)
        self.out_count = len(outputs)
        self.inputs = inputs
        self.outputs = outputs
        self.signature = sender.sign(json.dumps(self.to_dict(include_signature=False)))
        #version
        #lock_time
    
    @property
    def fee(self):
        
        total_in = sum(i.prev_tx.outputs[i.prev_txout_index].amount for i in self.inputs)
        total_out = sum(o.amount for o in self.outputs)

        return total_in - total_out
    

    def to_dict(self, include_signature=True):
        
        d = {
            "inputs": list(map(TransactionInput.to_dict, self.inputs)),
            "outputs": list(map(TransactionOutput.to_dict, self.outputs)),
        }
        
        if include_signature:
            d["signature"] = self.signature
        return d
    
    
    def tx_hash(self):
        return sha256(json.dumps(self.to_dict()))

In [280]:
class CoinbaseTransaction(Transaction):

    def __init__(self, address, amount):
        self.in_count = self.out_count = 1
        self.inputs = [
            
        ]
        self.outputs = [
            TransactionOutput(address, amount)
        ]
        self.signature = 'coinbase'
    
    @property
    def fee(self):
        return 0
        
    def to_dict(self, include_signature=False):
        return super().to_dict(include_signature=False)

In [281]:
### TEST (wallets, transactions)


alice = Wallet()
bob = Wallet()
walter = Wallet()

# This gives 25 coins to Alice
t1 = CoinbaseTransaction(alice.address, 25)

# Of those 25, Alice will spend
# Alice -- 5 --> Bob
#       -- 15 --> Alice
#       -- 5 --> Walter
t2 = Transaction(
    alice,
    [TransactionInput(t1, 0)],
    [TransactionOutput(bob.address, 5.0), TransactionOutput(alice.address, 15.0), TransactionOutput(walter.address, 5.0)]
)

# Walter -- 4.5 --> Bob
#           0.5 fee
t3 = Transaction(
    walter,
    [TransactionInput(t2, 2)],
    [TransactionOutput(bob.address, 4.5)])

# Bob -- 8 --> Walter
#     -- 1 --> Bob
#        0.5 fee
t4 = Transaction(
    bob,
    [TransactionInput(t2, 0), TransactionInput(t3, 0)],
    [TransactionOutput(walter.address, 8.0), TransactionOutput(bob.address, 1.0)]
)

#invalid transactions:

tx_double_spending = Transaction(
    bob,
    [TransactionInput(t2, 0), TransactionInput(t3, 0)],
    [TransactionOutput(walter.address, 8.0), TransactionOutput(bob.address, 1.0)]
)

tx_no_input = Transaction(
    bob,
    [],
    [TransactionOutput(walter.address, 8.0), TransactionOutput(bob.address, 1.0)]
)

tx_negative_amount = Transaction(
    bob,
    [TransactionInput(t2, 0), TransactionInput(t3, 0)],
    [TransactionOutput(walter.address, -8.0), TransactionOutput(bob.address, 1.0)]
)

tx_overspending = Transaction(
    bob,
    [TransactionInput(t2, 0), TransactionInput(t3, 0)],
    [TransactionOutput(walter.address, 430.0), TransactionOutput(bob.address, 1.0)]
)

tx_not_signed = Transaction(
    alice,
    [TransactionInput(t2, 0), TransactionInput(t3, 0)],
    [TransactionOutput(walter.address, 8.0), TransactionOutput(bob.address, 1.0)]
)

tx_from_multiple_addresses = Transaction(
    bob,
    [TransactionInput(t1, 0), TransactionInput(t3, 0)],
    [TransactionOutput(walter.address, 8.0), TransactionOutput(bob.address, 1.0)]
)


transactions = [t1, t2, t3, t4]
walter.compute_balance(transactions)
t1.fee

0

These messages hold a single transaction. :

Check syntactic correctness

- DONE Make sure neither in or out lists are empty

-  - Size in bytes <= MAX_BLOCK_SIZE

- DONE Each output value, as well as the total, must be in legal money range

-  - Make sure none of the inputs have hash=0, n=-1 (coinbase transactions)

- Check that nLockTime <= INT_MAX[1], size in bytes >= 100[2], and sig opcount <= 2[3]

- Reject "nonstandard" transactions: scriptSig doing anything other than pushing numbers on the stack, or scriptPubkey 
 not matching the two usual forms[4]

- - Reject if we already have matching tx in the pool, or in a block in the main branch

- - For each input, if the referenced output exists in any other tx in the pool, reject this transaction.[5]

- - For each input, look in the main branch and the transaction pool to find the referenced output transaction. If the 
 output transaction is missing for any input, this will be an orphan transaction. Add to the orphan transactions, if a  matching transaction is not in there already.

- For each input, if the referenced output transaction is coinbase (i.e. only 1 input, with hash=0, n=-1), it must have at least COINBASE_MATURITY (100) confirmations; else reject this transaction

- - For each input, if the referenced output does not exist (e.g. never existed or has already been spent), reject this transaction[6]

- DONE:  Using the referenced output transactions to get input values, check that each input value, as well as the sum, are in legal money range

- DONE:  Reject if the sum of input values < sum of output values

- - Reject if transaction fee (defined as sum of input values minus sum of output values) would be too low to get into an empty block

- Verify the scriptPubKey accepts for each input; reject if any are bad

- Add to transaction pool[7]:

- "Add to wallet if mine"

- Relay transaction to peers

- For each orphan transaction that uses this one as one of its inputs, run all these steps (including this one) recursively on that orphan

    

In [271]:
class Node(object):
    
    def __init__(self):
        transaction_pool = []    #should be stored only in the blockchain
        mempool = []         #verified but uncomfirmed txs
        #unspent_txs = []    #should be in the wallets
    
    
    def relay_transaction(self, transaction):
        #Add to transaction pool[7]:

        #"Add to wallet if mine"

        #Relay transaction to peers
        pass

    
    def verify_transaction(self, transaction):
        
        tx_message = json.dumps(transaction.to_dict(include_signature=False))
        
        if isinstance(transaction, CoinbaseTransaction):
                # TODO: verify coinbase tx
            return True
        
      
        if (len(transaction.inputs)==0 or len(transaction.inputs)==0):
            logging.error("Transaction inputs or outputs are missing! ")
            return False
        
#not neccesssary:
        for txin in transaction.inputs:
            amount = txin.prev_tx.outputs[txin.prev_txout_index].amount
            if (amount <= 0):
                logging.error(
                    "One of the transaction inputs is invalid! %s" % 
                    (txin.prev_tx.outputs[txin.prev_txout_index].amount)
                )
                return False
#            
        for txout in transaction.outputs:
            if(txout.amount <= 0):
                logging.error("Transaction output invalid! %s" % (txout.amount))
                return False
        
        if(transaction.fee <= 0):
            logging.error("Transaction fee invalid! %s" %(transaction.fee))
            return False
        
        first_input_address = transaction.inputs[0].parent_output.address
        for txin in transaction.inputs[1:]:
            if txin.parent_output.address != first_input_address:
                logging.error(
                    "Transaction inputs belong to multiple wallets (%s and %s)" %
                    (txin.parent_output.address, first_input_address)
                )
                return False


        if not verify_signature(first_input_address, tx_message, transaction.signature):
            logging.error("Invalid signature!")
            return False
            
        return True
    
    def verify_block(self, block):
        pass
    
    def relay_block(self, block, blockchain):
        pass
    
    
    

In [282]:
n1 = Node()

#print("Coinbase:" , n1.verify_transaction(t1))     #passed
#print("0 fee: " , n1.verify_transaction(t2))       #passed
#print("Valid: " , n1.verify_transaction(t3))       #passed
print("tx_double_spending" , n1.verify_transaction(tx_double_spending))      #not passed
#print("tx_no_input" , n1.verify_transaction(tx_no_input))                   #passed
#print("tx_negative_amount" , n1.verify_transaction(tx_negative_amount))     #passed
#print("tx_overspending" , n1.verify_transaction(tx_overspending))           #passed
#print("tx_not_signed" , n1.verify_transaction(tx_not_signed))               #passed
#print("tx_from_multiple_addresses" , n1.verify_transaction(tx_from_multiple_addresses))       #passed


5.0
4.5
tx_double_spending True


In [226]:
class Block(object):

    def __init__():
        #self.version
        self.prev_block_h
        self.block_h
        self.height
        #self.merkle_root
        self.timestamp
        self.difficulty
        self.nonce
        self.txn_count
        self.tx_list[]
        


SyntaxError: invalid syntax (<ipython-input-226-d91de6662636>, line 13)

In [168]:
class Blockchain(object):
    
    def __init__():
        pass

In [169]:
class Miner(object):
    def __init__():
        pass